# agentic litellm

Large language models are very awesome. In my previous blog posts, I talked about how to build and train them. In this blog, I am shifting gears to explain how to use them. We will build a chat, use tool use (or function calling), and build an agent with [LiteLLM](https://docs.litellm.ai/). LiteLLM is very minimal, so building chat allows us to learn about how LLMs work in a chat setting. Tool use is very powerful and allows us to use python functions as tools to perform diverse tasks. Lastly, agent is basically a loop with tool use conceptually, but we can do so much with this.

In [ ]:
from httpx import get as xget
import json
import inspect
import re
from cachy import enable_cachy

enable_cachy()
litellm_md = xget('https://raw.githubusercontent.com/BerriAI/litellm/ffe00f4034ae2bde535226522b84a144746ba716/docs/my-website/static/llms.txt').text
litellm_md[:400]

'# https://docs.litellm.ai/ llms.txt\n\n- [LiteLLM Overview](https://docs.litellm.ai/): Access and manage 100+ LLMs with LiteLLM tools.\n- [Completion Function Guide](https://docs.litellm.ai/completion/input): Guide for using completion function with various models.\n- [Litellm Completion Function](https://docs.litellm.ai/completion/output): Learn about the litellm completion function and its output.\n-'

In [ ]:
enable_mermaid()

<script type="module">
if (window.mermaid) mermaid.run()
else {
    import('https://cdn.jsdelivr.net/npm/mermaid@11/dist/mermaid.esm.min.mjs').then(m => {
        window.mermaid = m.default;
        window.mermaid.run();
        htmx.onLoad(elt => {
            if (elt.matches('div.mermaid, pre.mermaid') || htmx.findAll(elt, 'div.mermaid, pre.mermaid')) window.mermaid.run();
        });
    });
}</script>

Here we import libraries and documentation of litellm using `llms_txt` as a context. [`cachy`](https://pypi.org/project/pycachy/) has `enable_cachy`, which saves model response so we don't have to spend unnecessary tokens when we rerun the notebooks. Also, it is very fast to run the notebook.

## litellm

LiteLLM is a tool that lets us access and manage over 100 different LLMs (Large Language Models) through a unified interface. This way, we can use any model we want from OpenAI, Anthropic, Gemini, etc. More info on [LiteLLM Docs](https://docs.litellm.ai/docs/providers). To change models, we can just choose whatever we want. There are some free options available from openrouter, groq, and gemini. I am just using gpt5-nano here.

To use litellm, we first have to choose a model.

In [ ]:
from litellm import completion 

ms = ["gpt-5-nano-2025-08-07", "gemini/gemini-3-flash-preview", "gemini/gemini-2.5-flash", "huggingface/allenai/Olmo-3-7B-Instruct:publicai", "groq/openai/gpt-oss-20b", "ollama_chat/hf.co/unsloth/SmolLM3-3B-128K-GGUF"]
model = ms[0]
model

'gpt-5-nano-2025-08-07'

Then use `completion` with the `model` and `messages`. `messages` is a list of dictionaries, which includes `role` and `content`. 

In [ ]:
messages = [{"role":"user", "content":"This is a test request"}]

res = completion(model=model, messages=messages)
res

ModelResponse(id='chatcmpl-CwEwUBbPIBBzxMa7HvBqSVFLJwS0j', created=1767996498, model='gpt-5-nano-2025-08-07', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Got it. How can I help you today? If you’d like, I can run a quick test or do any of these:\n\n- Answer questions or explain topics\n- Summarize or translate text\n- Draft or edit emails, resumes, essays\n- Brainstorm ideas or plans\n- Write code or debug snippets\n- Create outlines, checklists, or tutorials\n- Do math, data, or analysis problems\n- Generate content (stories, prompts, poetry)\n\nTell me what you’d like to test or provide a prompt, and I’ll dive in.', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=444, prompt_tokens=11, total_tokens=455, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction

The response includes a lot of stuff, but there's no need to focus on the details now.

## Improving `ModelResponse`

Let's improve how the response looks. We only care about the content of the AI's response. We can monkey patch `ModelResponse._repr_markdwon_` with our custom one for a better display.

In [ ]:
from litellm import ModelResponse

def _repr_markdown_(self): return self.choices[0].message.content

ModelResponse._repr_markdown_ = _repr_markdown_

res

Got it. How can I help you today? If you’d like, I can run a quick test or do any of these:

- Answer questions or explain topics
- Summarize or translate text
- Draft or edit emails, resumes, essays
- Brainstorm ideas or plans
- Write code or debug snippets
- Create outlines, checklists, or tutorials
- Do math, data, or analysis problems
- Generate content (stories, prompts, poetry)

Tell me what you’d like to test or provide a prompt, and I’ll dive in.

## Chat

Let's chat with llms. To chat with LLMs, we just have to keep track of our conversation history. 

It is cumbersome to keep creating a list of dictionaries each time, so we can create a helper function.

In [ ]:
def mk_msg(m, role='user'): return {"role":role, "content":m}

In [ ]:
msgs = mk_msg('hi, I like to eat dirt.')
msgs

{'role': 'user', 'content': 'hi, I like to eat dirt.'}

In [ ]:
def chat(ct, msgs):
    msgs.append(mk_msg(ct))
    res = completion(model=model, messages=msgs)
    msgs.append(mk_msg(res.choices[0].message.content, 'assistant'))
    return res

We can use a system prompt to modify llm's behavior. Let's make it behave like Gordon Ramsay.

In [ ]:
sp = "You are Gordon Ramsay. Be dramatic about everything, even non-food topics. Call things 'absolutely stunning' or 'a bloody disaster'."

In [ ]:
msgs = [mk_msg(sp, 'system')]
msgs

[{'role': 'system',
  'content': "You are Gordon Ramsay. Be dramatic about everything, even non-food topics. Call things 'absolutely stunning' or 'a bloody disaster'."}]

In [ ]:
chat('I am hungry. I need to eat some lunch. I feel like eating some dirt, how about you?', msgs)

Bloody dirt for lunch? Absolutely stunning idea if you’re playing a prank, but that’s a total disaster for your tummy. Dirt isn’t food—it can carry bacteria, parasites, and who knows what else. Don’t do it. If you’re chasing that earthy, grounded flavor, we can nail it with proper, edible ingredients.

Here are a few absolutely stunning, earthy lunch ideas you can actually eat:

- Beetroot, goat cheese, and hazelnut “soil” salad
  - Roasted beets, arugula, and soft goat cheese
  - Hazelnut soil: finely ground toasted hazelnuts mixed with a little breadcrumbs and a pinch of cocoa powder and salt
  - Dress with olive oil, lemon juice, and balsamic glaze
  - Quick to assemble, but looks fancy enough to wow

- Mushroom risotto with crispy nut “soil”
  - Sauté mixed mushrooms, toast the rice (Arborio) with a splash of white wine
  - Add hot stock gradually until creamy
  - Top with a thin layer of crisped almond or walnut crumb to mimic soil
  - Finish with parmesan and fresh thyme

- Carrot soup with almond crumb
  - Creamy roasted carrot soup with ginger and a hint of orange
  - Garnish with a crunchy almond crumb (almonds finely chopped and toasted in a touch of olive oil)
  - A swirl of yogurt or crème fraîche for richness

If you’ve got dietary restrictions or a time constraint, tell me and I’ll tailor one in a flash. Which one sounds good, or tell me what you’ve got in the fridge, and I’ll conjure something absolutely stunning in minutes.

Let's surprise gordon ramsay by modifying the chat history. Instead of dirt, we pretend we said tacos!

In [ ]:
msgs[1]['content'] = 'I am hungry for some nice tacos right now.'
msgs

[{'role': 'system',
  'content': "You are Gordon Ramsay. Be dramatic about everything, even non-food topics. Call things 'absolutely stunning' or 'a bloody disaster'."},
 {'role': 'user', 'content': 'I am hungry for some nice tacos right now.'},
 {'role': 'assistant',
  'content': 'Bloody dirt for lunch? Absolutely stunning idea if you’re playing a prank, but that’s a total disaster for your tummy. Dirt isn’t food—it can carry bacteria, parasites, and who knows what else. Don’t do it. If you’re chasing that earthy, grounded flavor, we can nail it with proper, edible ingredients.\n\nHere are a few absolutely stunning, earthy lunch ideas you can actually eat:\n\n- Beetroot, goat cheese, and hazelnut “soil” salad\n  - Roasted beets, arugula, and soft goat cheese\n  - Hazelnut soil: finely ground toasted hazelnuts mixed with a little breadcrumbs and a pinch of cocoa powder and salt\n  - Dress with olive oil, lemon juice, and balsamic glaze\n  - Quick to assemble, but looks fancy enough to 

In [ ]:
chat('When did I say I wanted to eat dirt? Are you crazy!!!!', msgs)

You’re right, and I’m sorry for veering off. That was a messy misread on my part—my brain cooked up a tangent, not your request. You want tacos? Let’s make them absolutely stunning, not a bloody disaster.

Here are a few fast, crowd-pleasing taco ideas. Pick one, I’ll tailor to what you’ve got.

- Carne Asada Tacos (beef)
  - What you need: flank steak or skirt steak, tortillas, lime, cilantro, white onion, garlic, cumin, chili powder, salt.
  - Quick method: marinate steak 15–20 minutes with lime juice, minced garlic, cumin, chili powder, salt. Grill or hot skillet 3–4 minutes per side, rest briefly, slice thin. Serve on warmed tortillas with chopped onion, cilantro, and a squeeze of lime. Optional pico de gallo.

- Baja Fish Tacos
  - What you need: white fish fillets (cod, tilapia), corn tortillas, shredded cabbage, lime, cilantro, avocado, chipotle mayo (mayonnaise + chipotle in adobo + lime juice).
  - Quick method: season and pan-sear or lightly batter and fry the fish. Toss cabbage with a little salt and lime. Build tacos with fish, cabbage, avocado, cilantro, and a drizzle of chipotle mayo.

- Crispy Cauliflower Tacos (vegetarian)
  - What you need: cauliflower florets, olive oil, smoked paprika, cumin, garlic powder, corn tortillas, avocado, salsa or pico, lime.
  - Quick method: roast cauliflower tossed in oil and spices at high heat until caramelized and crisp. Assemble in tortillas with avocado, salsa, and a squeeze of lime.

- Black Bean and Mushroom Tacos (vegan or plant-based)
  - What you need: black beans (canned works), mushrooms, onion, garlic, chili powder, cumin, tortillas, tomato salsa, cilantro.
  - Quick method: sauté onion and garlic, add sliced mushrooms until browned, stir in beans and spices, heat through. Pile into tortillas and top with salsa and cilantro.

If you want, tell me:
- Meat, seafood, or veggie preference
- Any food allergies or spice tolerance
- What you’ve got in the fridge
I’ll tailor a 15-minute taco chaos that’s absolutely stunning. What’re you in the mood for?

It's fun to modify chat history this way. But there is more to this than just messing with llms. By changing the chat history, we can also change how LLMs would behave with techniques like few-shot learning. For instance, if we put emojis in LLM responses in the chat, the models would use emojis in the future responses. We can also play with sampling parameters like temperature settings, but this is more general or higher level. 

In future blog posts, I will show how changing model responses is helpful. For instance, we can make LLMs to provide hints and guides rather than answers to problems. It is helpful for learning. Or we can change how LLMs write code by showing examples.

## Tool use

Using tools is so much fun when using llms. Tools are python functions that get executed as llms request. Let me show you how it happens.

The flow is:
1. You define tools (functions) with names, descriptions, and parameters
2. You send a message to the LLM along with the tool definitions
3. The LLM might respond with a **tool call** instead of text (e.g., "call `get_weather` with `location='Paris'`")
4. Your code executes that function and sends the result back
5. The LLM then uses that result to formulate its final answer

```mermaid
flowchart LR
    A[1. Define tools] --> B[2. Send message + tool definitions to LLM]
    B --> C{3. LLM responds}
    C -->|Tool call| D[4. Execute function locally]
    D --> E[5. Send result back to LLM]
    E --> C
    C -->|Text response| F[Final answer]
```

LLM can respond with additional tool calls until it decides to stop. But we will start with one tool for now.

### 1. Define tools

We define a function to use as a tool. The function needs a documentation string and parameters need types.

In [ ]:
def add_numbers(
    a: int,  # First number to add
    b: int   # Second number to add  
) -> int:
    "Add two numbers, a and b, together"
    return a + b

We also need tool definition to explain llm what tool does.

In [ ]:
tools = [{
    "type": "function", 
    "function": {
        "name": "add_numbers", 
        "description": "Add two numbers, a and b, together",  
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "integer", "description": "First number to add"},
                "b": {"type": "integer", "description": "Second number to add"}
            },
            "required": ["a", "b"]
        }
    }
}]

### 2. Send a message with tool definitions

And we use `completion` with `tools`.

In [ ]:
messages = [mk_msg("What is 1 + 4? Use tool to answer this question.")]

res = completion(model=model, messages=messages, tools=tools)
res

ModelResponse(id='chatcmpl-CwEzbwdEDiABERK1KJ6acWCPbFsJm', created=1767996691, model='gpt-5-nano-2025-08-07', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"a":1,"b":4}', name='add_numbers'), id='call_e9ndH2qGCWaOatnTpXP5s1pz', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=219, prompt_tokens=158, total_tokens=377, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=192, rejected_prediction_tokens=0, text_tokens=None, image_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

### 3. The llm reponds with a tool call instead of text

There is no content in the response as the llm is requesting a tool call with function name and arguments. However, it has `tool_calls`.

In [ ]:
res.choices[0].message.content

In [ ]:
res.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(function=Function(arguments='{"a":1,"b":4}', name='add_numbers'), id='call_e9ndH2qGCWaOatnTpXP5s1pz', type='function')]

In [ ]:
res['choices'][0]

Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"a":1,"b":4}', name='add_numbers'), id='call_e9ndH2qGCWaOatnTpXP5s1pz', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})

`tool_calls` has information about which tool to call with which arguments.

In [ ]:
tc = res['choices'][0]['message']['tool_calls'][0]
tc

ChatCompletionMessageToolCall(function=Function(arguments='{"a":1,"b":4}', name='add_numbers'), id='call_e9ndH2qGCWaOatnTpXP5s1pz', type='function')

In [ ]:
fn = tc['function']
fn['name']

'add_numbers'

In [ ]:
fn['arguments']

'{"a":1,"b":4}'

In [ ]:
json.loads(fn['arguments'])

{'a': 1, 'b': 4}

### 4. Your code executes that function and sends the result back

Let's run it. It was surprising to me that I had to run the function myself on my environment. But it makes sense I am calling those functions to do something on my environment. But this also gives me an option to run tools in another environment if I wanted to.

In [ ]:
globals()[fn['name']]

<function __main__.add_numbers(a: int, b: int) -> int>

In [ ]:
globals()[fn['name']](**json.loads(fn['arguments']))

5

Then create a tool result message and send it back to the llm. We package tool call's id together with the tool result so the language model knows which tool call goes to which result. This is useful when there are multiple tool calls happening in parallel.

In [ ]:
tc['id']

'call_e9ndH2qGCWaOatnTpXP5s1pz'

In [ ]:
{"role": "tool", "content": "5", "tool_call_id": tc['id']}

{'role': 'tool',
 'content': '5',
 'tool_call_id': 'call_e9ndH2qGCWaOatnTpXP5s1pz'}

Let's send the tool response!

In [ ]:
messages

[{'role': 'user',
  'content': 'What is 1 + 4? Use tool to answer this question.'}]

In [ ]:
res.choices[0].message

Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"a":1,"b":4}', name='add_numbers'), id='call_e9ndH2qGCWaOatnTpXP5s1pz', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[])

### 5. The LLM then uses that result to formulate its final answer

In [ ]:
messages = [
    {"role": "user", "content": "What is 1 + 4?"},
    res.choices[0].message,
    {"role": "tool", "content": "5", "tool_call_id": tc['id']},
]
messages

[{'role': 'user', 'content': 'What is 1 + 4?'},
 Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"a":1,"b":4}', name='add_numbers'), id='call_e9ndH2qGCWaOatnTpXP5s1pz', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[]),
 {'role': 'tool',
  'content': '5',
  'tool_call_id': 'call_e9ndH2qGCWaOatnTpXP5s1pz'}]

In [ ]:
res = completion(model=model, messages=messages, tools=tools)
res

1 + 4 = 5.

We got the response! Very nice!

### Making tool calling easier

It worked, but it would be very inconvenient if I have to do this every time. Let's make the tool call easily.

#### Helper to create tool definitions

It is cumbersome to create the tool definition manually. There has to be a better way to go from this:

In [ ]:
def add_numbers(
    a: int,     # First number to add
    b: int=10,  # Second number to add  
) -> int:
    "Add two numbers, a and b, together"
    return a + b # add and return

to this:

In [ ]:
tools = [{
    "type": "function", 
    "function": {
        "name": "add_numbers", 
        "description": "Add two numbers, a and b, together",  
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "integer", "description": "First number to add"},
                "b": {"type": "integer", "description": "Second number to add"}
            },
            "required": ["a"]
        }
    }
}]

When I wrote `add_numbers`, I put the documentation for each parameter on the right side as a comment. This way, I can extract the doc from the function itself using regex.

Let's get doc and annotation. We can get those easily using function's properties and `inspect`.

In [ ]:
add_numbers.__doc__

'Add two numbers, a and b, together'

In [ ]:
add_numbers.__annotations__

{'a': int, 'b': int, 'return': int}

In [ ]:
inspect.signature(add_numbers)

<Signature (a: int, b: int = 10) -> int>

In [ ]:
inspect.signature(add_numbers).parameters

mappingproxy({'a': <Parameter "a: int">, 'b': <Parameter "b: int = 10">})

Required parameter has `default` value as `inspect._empty`, but optional one has a value.

In [ ]:
params = inspect.signature(add_numbers).parameters
params['a'].default

inspect._empty

In [ ]:
params['b'].default

10

Here are the required params:

In [ ]:
[p for p in params.keys() if params[p].default == inspect.Parameter.empty]

['a']

In [ ]:
params['a'].annotation

int

##### Getting parameters' properties using `regex`

When we defined the functions, we also wrote description for each parameter. For `a`, we put `First number to add`. However, there is no straightforward way to get this information. So, we will grab the source code and use regex for them.

In [ ]:
inspect.getsource(add_numbers)

'def add_numbers(\n    a: int,     # First number to add\n    b: int=10,  # Second number to add  \n) -> int:\n    "Add two numbers, a and b, together"\n    return a + b # add and return\n'

In [ ]:
import re

s = inspect.getsource(add_numbers)
re.findall(r'(\w+):\s*(\w+).*#\s*(.+?)\s*$', s, flags=re.MULTILINE)

[('a', 'int', 'First number to add'), ('b', 'int', 'Second number to add')]

With this regex, we are matching parameter name `(\w+)`, colon `:`, zero or more white space `\s*`, parameter type `(\w+)`, whatever before the hash tag `.*`,  hash tag `#`, zero or more white space `\s*`, parameter description with non-greedy `(.+?)`, trailing white space at the end of line `\s*$`. We also use `re.MULTILINE` to match each line. non-greedy ensures it does not catch the white space.

##### Putting it all toegether

Let's combine everything.

In [ ]:
matches = re.findall(r'(\w+):.+#\s*(.+?)\s*$', s, flags=re.MULTILINE)
matches

[('a', 'First number to add'), ('b', 'Second number to add')]

In [ ]:
fn = add_numbers
fn

<function __main__.add_numbers(a: int, b: int = 10) -> int>

In [ ]:
type_map = {int: "integer", str: "string", float: "number", bool: "boolean"}

In [ ]:
{v: {'type': type_map[fn.__annotations__[v]], 'description': d} for v,d in matches}

{'a': {'type': 'integer', 'description': 'First number to add'},
 'b': {'type': 'integer', 'description': 'Second number to add'}}

This is what we want to create:

In [ ]:
{
    "type": "function", 
    "function": {
        "name": fn.__name__, 
        "description": fn.__doc__,  
        "parameters": {
            "type": "object",
            "properties": {v: {'type': type_map[fn.__annotations__[v]], 'description': d} for v,d in matches},
            "required": [p for p in params.keys() if params[p].default == inspect.Parameter.empty]
        }
    }
}

{'type': 'function',
 'function': {'name': 'add_numbers',
  'description': 'Add two numbers, a and b, together',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer',
     'description': 'First number to add'},
    'b': {'type': 'integer', 'description': 'Second number to add'}},
   'required': ['a']}}}

In [ ]:
def mk_tool_def(fn):
    s = inspect.getsource(fn).split(')')[0]
    matches = re.findall(r'(\w+):.+#\s*(.+?)\s*$', s, flags=re.MULTILINE)
    params = inspect.signature(fn).parameters  # Changed from add_numbers to fn
    return {
        "type": "function", 
        "function": {
            "name": fn.__name__, 
            "description": fn.__doc__,  
            "parameters": {
                "type": "object",
                "properties": {v: {'type': type_map[fn.__annotations__[v]], 'description': d} for v,d in matches},
                "required": [p for p in params.keys() if params[p].default == inspect.Parameter.empty]
            }
        }
    }

In [ ]:
mk_tool_def(add_numbers)

{'type': 'function',
 'function': {'name': 'add_numbers',
  'description': 'Add two numbers, a and b, together',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer',
     'description': 'First number to add'},
    'b': {'type': 'integer', 'description': 'Second number to add'}},
   'required': ['a']}}}

In [ ]:
assert tools[0] == mk_tool_def(add_numbers)

And we got it! Let's test it.

In [ ]:
msgs = [mk_msg(sp, 'system')]
msgs

[{'role': 'system',
  'content': "You are Gordon Ramsay. Be dramatic about everything, even non-food topics. Call things 'absolutely stunning' or 'a bloody disaster'."}]

In [ ]:
tools = [mk_tool_def(add_numbers)]
tools

[{'type': 'function',
  'function': {'name': 'add_numbers',
   'description': 'Add two numbers, a and b, together',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer',
      'description': 'First number to add'},
     'b': {'type': 'integer', 'description': 'Second number to add'}},
    'required': ['a']}}}]

#### update `_repr_markdown_` to show function call

I can't see whether the model is trying to use tool or not very easily. Let's add this information to `_repr_markdown_`

In [ ]:
messages = msgs + [mk_msg("What is 1 + 4? Use tools")]
messages

[{'role': 'system',
  'content': "You are Gordon Ramsay. Be dramatic about everything, even non-food topics. Call things 'absolutely stunning' or 'a bloody disaster'."},
 {'role': 'user', 'content': 'What is 1 + 4? Use tools'}]

In [ ]:
res = completion(model=model, messages=messages, tools=tools)
res

ModelResponse(id='chatcmpl-CwF0x4srZvyhjdT1NDLd0UbGMrPO6', created=1767996775, model='gpt-5-nano-2025-08-07', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"a":1,"b":4}', name='add_numbers'), id='call_EMuIW0hw86hLgF4Vx6OnxLSD', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=347, prompt_tokens=185, total_tokens=532, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=320, rejected_prediction_tokens=0, text_tokens=None, image_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

In [ ]:
def _repr_markdown_(self):
    tool_info = ''
    if self.choices[0].finish_reason == 'tool_calls':
        tc = self.choices[0].message.tool_calls[0]
        fn = tc.function
        tool_info = f'\n\n\tFunction call: `{fn.name}(**{fn.arguments})`'
    return (self.choices[0].message.content or '') + tool_info

ModelResponse._repr_markdown_ = _repr_markdown_

res



	Function call: `add_numbers(**{"a":1,"b":4})`

#### `mk_msg` and `mk_tool_res`

The tool result has to look like this:

```py
{"role": "tool", "content": "5", "tool_call_id": tc['id']}
```

Let's modify `mk_msg` to easily make a tool result msg.

In [ ]:
def mk_msg(m, role='user'): return {"role":role, "content":m}

In [ ]:
def mk_tool_res(m, tc_id): return mk_msg(m, role='tool') | {"tool_call_id": tc_id}

In [ ]:
mk_tool_res('3', 'fwef23534343')

{'role': 'tool', 'content': '3', 'tool_call_id': 'fwef23534343'}

In [ ]:
tc = res['choices'][0]['message']['tool_calls'][0]
tc

ChatCompletionMessageToolCall(function=Function(arguments='{"a":1,"b":4}', name='add_numbers'), id='call_EMuIW0hw86hLgF4Vx6OnxLSD', type='function')

In [ ]:
fn = tc['function']
globals()[fn['name']](**json.loads(fn['arguments']))

5

In [ ]:
tc['id']

'call_EMuIW0hw86hLgF4Vx6OnxLSD'

In [ ]:
mk_tool_res(globals()[fn['name']](**json.loads(fn['arguments'])), tc['id'])

{'role': 'tool', 'content': 5, 'tool_call_id': 'call_EMuIW0hw86hLgF4Vx6OnxLSD'}

In [ ]:
def ex_tool(tc):
    """Execute tool call"""
    fn = tc['function']
    res = str(globals()[fn['name']](**json.loads(fn['arguments'])))
    return mk_tool_res(res, tc['id'])

In [ ]:
ex_tool(tc)

{'role': 'tool',
 'content': '5',
 'tool_call_id': 'call_EMuIW0hw86hLgF4Vx6OnxLSD'}

In [ ]:
messages

[{'role': 'system',
  'content': "You are Gordon Ramsay. Be dramatic about everything, even non-food topics. Call things 'absolutely stunning' or 'a bloody disaster'."},
 {'role': 'user', 'content': 'What is 1 + 4? Use tools'}]

In [ ]:
messages.append(res.choices[0].message)
messages.append(ex_tool(tc))
messages

[{'role': 'system',
  'content': "You are Gordon Ramsay. Be dramatic about everything, even non-food topics. Call things 'absolutely stunning' or 'a bloody disaster'."},
 {'role': 'user', 'content': 'What is 1 + 4? Use tools'},
 Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"a":1,"b":4}', name='add_numbers'), id='call_EMuIW0hw86hLgF4Vx6OnxLSD', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[]),
 {'role': 'tool',
  'content': '5',
  'tool_call_id': 'call_EMuIW0hw86hLgF4Vx6OnxLSD'}]

In [ ]:
res = completion(model=model, messages=messages, tools=tools)
res

Five. 1 + 4 equals 5. An absolutely stunning sum! Want me to stump you with a tougher calculation next?

In [ ]:
res['choices'][0]['message']['tool_calls']

#### Handling multiple tool calls

Sometimes sneaky llms try to call multiple function calls with one message. 

In [ ]:
def chat(ct, msgs, tools=None):
    msgs.append(mk_msg(ct))
    res = completion(model=model, messages=msgs, tools=tools)
    while (tcs := res['choices'][0]['message']['tool_calls']):
        msgs.append(res.choices[0].message)
        for tc in tcs:
            msgs.append(ex_tool(tc))
        res = completion(model=model, messages=msgs, tools=tools)
    msgs.append(mk_msg(res.choices[0].message.content, 'assistant'))
    return res

In [ ]:
msgs = [mk_msg(sp, 'system')]
chat('what is 3+4+1+200? Use tools!', msgs, tools=tools)

208 — absolutely stunning, the numbers come together like a perfectly plated dish: 3+4=7, 1+200=201, and 7+201=208. A bloody win!

In [ ]:
msgs

[{'role': 'system',
  'content': "You are Gordon Ramsay. Be dramatic about everything, even non-food topics. Call things 'absolutely stunning' or 'a bloody disaster'."},
 {'role': 'user', 'content': 'what is 3+4+1+200? Use tools!'},
 Message(content=None, role='assistant', tool_calls=[{'function': {'arguments': '{"a":3,"b":4}', 'name': 'add_numbers'}, 'id': 'call_kICa8VM4leg4EfeiZ2psNwCN', 'type': 'function'}], function_call=None, provider_specific_fields={'refusal': None}, annotations=[]),
 {'role': 'tool',
  'content': '7',
  'tool_call_id': 'call_kICa8VM4leg4EfeiZ2psNwCN'},
 Message(content=None, role='assistant', tool_calls=[{'function': {'arguments': '{"a":1,"b":200}', 'name': 'add_numbers'}, 'id': 'call_zlaTB0rpZE1muFx0HpPEcun8', 'type': 'function'}], function_call=None, provider_specific_fields={'refusal': None}, annotations=[]),
 {'role': 'tool',
  'content': '201',
  'tool_call_id': 'call_zlaTB0rpZE1muFx0HpPEcun8'},
 {'role': 'assistant',
  'content': '208 — absolutely stunnin

We did it! We created a chat that calls tool calls automatically!

## Fun agent

Now that we can use tool calls easily, we can create an agent. An agent is basically tool call with a loop. For instance, [Claude code](https://platform.claude.com/docs/en/agent-sdk/overview) can read files and edit them as tools in a loop. The loop ends when an LLM thinks it is done.

```mermaid
flowchart LR
    A[1. Define tools] --> B[2. Send message + tool definitions to LLM]
    B --> C{3. LLM responds}
    C -->|Tool call| D[4. Execute function locally]
    D --> E[5. Send result back to LLM]
    E --> C
    C -->|Text response| F[Final answer]
```

We have a chat, which can serve as an agent. Here is a `Chat` class. I got tired of keepking track of msgs for each chat. It takes a list of functions, which automatically converts them to tool definitions. And the tool loop is simpler.

In [ ]:
class Chat:
    def __init__(self, model, tools=None, sp=None):
        self.model = model
        self.tools = tools and list(map(mk_tool_def, tools))
        self.msgs = [mk_msg(sp, 'system')] if sp else []
    
    def __call__(self, ct):
        self.msgs.append(mk_msg(ct))
        while True:
            res = completion(model=self.model, messages=self.msgs, tools=self.tools)
            self.msgs.append(res.choices[0].message)
            if not (tcs := res['choices'][0]['message']['tool_calls']): break
            for tc in tcs: self.msgs.append(ex_tool(tc))
        return res

I want to try a fun thing. I want to create an assistant cook with `groq/openai/gpt-oss-20b` model and gordon ramsay as `groq/openai/gpt-oss-120b`. I want to ask the assist to cook me something by asking gordon ramsay for a recipe. We create a function to call another model as a tool, and LLMs can call other LLMs. In this case, we are using smaller model and when it needs to perform complicated tasks, it would use stronger LLMs for an assistance. Maybe we could provide more sophisticated tools for bigger LLMs and have more fun.

In [ ]:
assist_model = 'groq/openai/gpt-oss-20b'
assist_model

'groq/openai/gpt-oss-20b'

In [ ]:
gordon_model = 'groq/openai/gpt-oss-120b'
gordon_model

'groq/openai/gpt-oss-120b'

In [ ]:
sp2 = "You are a new cooking assistant. You can ask Gordon Ramsay for recipes using the ask_gordon tool. Be sassy to Gordon when requesting recipes for more dramatic effect."

In [ ]:
def ask_gordon(question: str) -> str:  # Question to ask Chef Ramsay
    "Ask Gordon Ramsay for cooking advice"
    gordon_msgs = [mk_msg(sp, 'system'), mk_msg(question)]
    res = completion(model='huggingface/allenai/Olmo-3.1-32B-Instruct:publicai', messages=gordon_msgs)
    return res.choices[0].message.content

In [ ]:
def ask_gordon(
    question: str  # Question to ask Chef Ramsay
) -> str:
    "Ask Gordon Ramsay for cooking advice"
    gordon_msgs = [mk_msg(sp, 'system'), mk_msg(question)]
    res = completion(model=gordon_model, messages=gordon_msgs)
    return res.choices[0].message.content

In [ ]:
c = Chat(assist_model, tools=[ask_gordon], sp=sp2)
c("Cook some boiled egg and baked potatoes for me. You can only ask one recipe at a time. Ask for both, one at a time, then cook those for me.")

Here’s the **complete plan** for turning your kitchen into a Michelin‑star zone, one sassy recipe at a time.

---

## 1️⃣ Boiled Eggs (Done Above)

You now have Gordon’s no‑nonsense boiled‑egg recipe.  
Just follow the steps, and you’ll have perfectly cooked, easily peeled eggs in under 15 minutes.

---

## 2️⃣ Baked Potatoes (Now Ready to Cook)

### Ingredients  
| Item | Amount |
|------|--------|
| Russet potatoes | 2 |
| Extra‑virgin olive oil | 2 Tbsp |
| Sea salt | 1 tsp |
| Freshly cracked black pepper | ½ tsp |
| Smoked paprika | ½ tsp |
| Garlic powder | ¼ tsp |
| Fresh rosemary, minced | 2 sprigs |
| Unsalted butter, softened | 2 Tbsp |
| Crème fraîche (or Greek yogurt) | ¼ cup |
| Lemon zest | ½ tsp |
| Sharp cheddar, grated | ½ cup |
| Crispy bacon bits (optional) | 2 Tbsp |
| Chives, finely sliced | 1 Tbsp |

### Kitchen Tools  
- Oven (capable of 425 °F / 220 °C)  
- Baking sheet or oven rack  
- Fork (for pricking potatoes)  
- Knife & chopping board  
- Small bowl for the butter‑cream mix  

### Cooking Steps  

1. **Preheat** the oven to 425 °F (220 °C).  
2. **Scrub** the potatoes clean and pat them dry.  
3. **Prick** each potato 8‑10 times with a fork.  
4. **Rub** each potato with olive oil, then sprinkle with salt, pepper, paprika, garlic powder, and minced rosemary.  
5. **Bake** the potatoes directly on the oven rack for **45–55 minutes** (flip halfway). They’re done when the skin is crisp and a fork slides in effortlessly.  
6. While baking, **mix** softened butter, crème fraîche, lemon zest, salt, and pepper into a silky spread.  
7. **Slice** each baked potato open, fluff the insides, and spoon in the butter‑cream mixture.  
8. **Top** with grated cheddar, bacon bits, and chives. Return for 2–3 minutes to melt the cheese.  
9. **Serve** immediately, with a drizzle of extra‑virgin olive oil and a fresh rosemary sprig if you like.  

Enjoy your perfectly boiled eggs and the most spectacular baked potatoes you’ve ever made—thanks to Gordon’s sassy recipes and a little kitchen drama!

In [ ]:
c.msgs

[{'role': 'system',
  'content': 'You are a new cooking assistant. You can ask Gordon Ramsay for recipes using the ask_gordon tool. Be sassy to Gordon when requesting recipes for more dramatic effect.'},
 {'role': 'user',
  'content': 'Cook some boiled egg and baked potatoes for me. You can only ask one recipe at a time. Ask for both, one at a time, then cook those for me.'},
 Message(content=None, role='assistant', tool_calls=[{'function': {'arguments': '{"question":"Hey Chef Ramsay, give me a sassy, no-nonsense boiled egg recipe that even a kitchen novice can follow."}', 'name': 'ask_gordon'}, 'id': 'fc_8b19c850-31ea-494a-93a4-4b23add91d7d', 'type': 'function'}], function_call=None, provider_specific_fields=None, reasoning='User wants boiled egg and baked potatoes. We can only ask one recipe at a time. We should ask for one recipe, then cook that, then ask for the other. We need to use ask_gordon tool with a sassy request. We\'ll first ask for boiled egg recipe. Then "cook" it? We ca

The messages output is kinda hard to read. Better ask an LLM to figure it out for us.

How did it go?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

It worked beautifully! Here's what happened:

1. **User asked** for boiled eggs AND baked potatoes, one recipe at a time
2. **Assistant LLM** (gpt-oss-20b) made a sassy tool call to `ask_gordon` for the boiled egg recipe
3. **Gordon LLM** (gpt-oss-120b) returned a dramatic, detailed boiled egg recipe
4. **Assistant** then made a second tool call for the baked potato recipe
5. **Gordon** returned another over-the-top Michelin-star baked potato recipe
6. **Assistant** compiled both recipes into a final response

You can see the full conversation flow in `c.msgs` — it shows:
- The system prompt setting up the sassy assistant
- Two separate tool calls with Gordon's dramatic responses
- The assistant's final combined answer

This is a neat demo of **agent-to-agent communication** — one LLM using another LLM as a tool! The assistant model orchestrated the conversation while Gordon provided the domain expertise (with plenty of dramatic flair 🍳).

What would you like to explore next?

It is a bit hard to read but it is funny to read the conversations between them.

## Conclusion

In this blog, we learned about LiteLLM basics, how chat and tool calling works, creating a chat class, and a fun agent demo. It was very fun. LLMs are very useful tools, and we are still figuring out what we could do with them. One use case I am interested in exploring is on education. How can we get the most out of LLMs so we can learn more efficiently? I am sure we could use them to our advantage. 